# B-SOSE preliminary

This notebook contains some early drafts and examples of interacting with a future BSOSE index on Argovis. None of the features or schema below are in production, and all may change before release.

In [ ]:
import requests, copy, os, sys, pandas, xarray
sys.path.append(os.path.abspath(os.pardir))
import helpers
from argovisHelpers import helpers as avh
import matplotlib.pyplot as plt

API_ROOT="https://argovis-apix-atoc-argovis-dev.apps.containers02.colorado.edu"

Start by grabbing some BSOSE data. It is timeseries formatted, each data document corresponding to a specific lat / long / depth triple, with metadata unique to this coordinate included on the data doc.

In [ ]:
qs = {
    "box": '[[-1,-31],[0,-29]]',
    "data": 'all'
}

bsose = requests.get(API_ROOT+"/timeseries/bsose", params=qs).json()

In [ ]:
bsose[0]

We can grab the corresponding metadata, one metadata doc for each lat/long point:

In [ ]:
qs_meta = copy.deepcopy(qs)
qs_meta['batchmeta'] = True
bsose_meta = requests.get(API_ROOT+"/timeseries/bsose", params=qs_meta).json()
bsose_meta[0]

From here it is a trivial matter to turn Argovis data into pandas data frames and xarray data sets, and leverage all the built in slicing and plotting those tools offer:

In [ ]:
df = helpers.level_df(bsose, 
                      ['TRAC04', 'SALT', 'longitude', 'latitude', 'level'], 
                      timesteps=bsose_meta[0]['timeseries'], 
                      index=["latitude","longitude","timestamp","level"]
                     )
ds = df.to_xarray()

In [ ]:
ds

Let's start by plotting a simple map of both our variables for a single timestamp and level:

In [ ]:
simplemap = ds.loc[{"timestamp":avh.parsetime('2012-12-31T00:00:00.000Z'), "level":ds['level'][0]}]
simplemap['TRAC04'].plot()

In [ ]:
simplemap['SALT'].plot()

We can also do timeseries for a specific geospatial point, to reveal seasonal cycles:

In [ ]:
simple_ts = ds.loc[{"longitude": ds['longitude'][0], "latitude": ds['latitude'][0], "level": ds['level'][0]}]
simple_ts['TRAC04'].plot()

In [ ]:
simple_ts['SALT'].plot()

Profiles at a given lat / long / timestep triple are constructed similarly:

In [ ]:
simple_profile = ds.loc[{"longitude": ds['longitude'][0], "latitude": ds['latitude'][0], "timestamp": ds['timestamp'][0]}]

In [ ]:
var = 'TRAC04'
plt.plot(simple_profile[var], simple_profile['level'])
plt.gca().invert_yaxis()
plt.xlabel(var)
plt.ylabel('depth')
plt.show()

In [ ]:
var = 'SALT'
plt.plot(simple_profile[var], simple_profile['level'])
plt.gca().invert_yaxis()
plt.xlabel(var)
plt.ylabel('depth')
plt.show()

BSOSE reports model cell areas at each latitude / longitude point. We can use these cell areas to construct area weighted means, including meridional and zonal means:

In [ ]:
weight_table = [{'weight': x['cell_area'], 'longitude': x['longitude'], 'latitude': x['latitude']} for x in bsose_meta]
weight_df = pandas.DataFrame(weight_table)
weight_ds = xarray.Dataset.from_dataframe(weight_df.set_index(["latitude", "longitude"]))
weighted_ds = ds.weighted(weight_ds["weight"])

In [ ]:
awm = weighted_ds.mean(("longitude", "latitude"))
awm['TRAC04'].plot(y="level",yincrease=False)

In [ ]:
awm['SALT'].plot(y="level",yincrease=False)

In [ ]:
meridional = weighted_ds.mean(("latitude")).loc[{"level":ds['level'][0]}]
meridional['TRAC04'].plot(y='timestamp')

In [ ]:
meridional['SALT'].plot(y='timestamp')

In [ ]:
zonal = weighted_ds.mean(("longitude")).loc[{"level":ds['level'][0]}]
zonal['TRAC04'].plot(y='timestamp')

In [ ]:
zonal['SALT'].plot(y='timestamp')

In [ ]:
x = {'geolocation': {'type': 'Point', 'coordinates': [10,10]}, 'level':99, 'timeseries': [1,2,3], 'data':[[10,11,12],[13,14,15]]}
y = {'geolocation': {'type': 'Point', 'coordinates': [20,20]}, 'level':99, 'timeseries': [5,6,7], 'data':[[20,21,22],[23,24,25]]}
z = {'geolocation': {'type': 'Point', 'coordinates': [30,30]}, 'level':99, 'timeseries': [5,6,7], 'data':[[200,210,220],[230,240,250]]}
X = {'geolocation': {'type': 'Point', 'coordinates': [10,10]}, 'level':99, 'timeseries': [1,2,3], 'data':[[10,11,12],[13,14,15]]}
Y = {'geolocation': {'type': 'Point', 'coordinates': [20,20]}, 'level':99, 'timeseries': [5,6,7], 'data':[[20,21,22],[23,24,25]]}
Z = {'geolocation': {'type': 'Point', 'coordinates': [30,30]}, 'level':98, 'timeseries': [5,6,7], 'data':[[200,210,220],[230,240,250]]}
x.get('level') == y.get('level')

In [ ]:
x['timeseries'] += y.get('timeseries', [])
x

In [ ]:
def combine_data_lists(lists):
    combined_list = []
    for sublists in zip(*lists):
        combined_sublist = []
        for sublist in sublists:
            combined_sublist.extend(sublist)
        combined_list.append(combined_sublist)
    return combined_list

combine_data_lists([x['data'],y['data'],z['data']])

In [ ]:
def combine_dicts(list1, list2):
    combined_list = []
    for dict1 in list1:
        combined = False
        for dict2 in list2:
            if dict1.get('geolocation') == dict2.get('geolocation') and dict1.get('level') == dict2.get('level'):
                combined_dict = dict1.copy()
                combined_dict['timeseries'] += dict2.get('timeseries', [])
                combined_dict['data'] = combine_data_lists([dict1.get('data', []), dict2.get('data', [])])
                combined_list.append(combined_dict)
                combined = True
                list2.remove(dict2)  # Remove combined element from list2
                break
        if not combined:
            combined_list.append(dict1)
    combined_list.extend(list2)  # Append remaining elements from list2
    return combined_list

In [ ]:
combine_dicts([x,y,z],[X,Y,Z])